<a href="https://colab.research.google.com/github/nidaa-hub/HaifaUniv_AI_Lab_2022/blob/main/AI_2022_Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***README***

to run this program please change the parameters in the main function (__name__ == "__main__")

comment out the functions/methodes that you want to use 

1.   remove comment out (#) from the function/methode that you want to use (in exampl: (calcPopulationFitness(population) or bullsEye(population))
2.   in line 222 "*mate*" choose the selection methode (Tournement/RWS/etc..) and (single/two/unified)CrossOver
1.   Press ▶ or ctrl+Enter

we added the following options to use in main the same way as (1):

      - randomImmigrants(population)
      - geneticDiversity(population)
      - thresholdSpeciation(population)
      - clustringSpeciation(population)
      - print - levenshtein distance
      - print - kendalltau distance

*   ID Submetter1: 316151232
*   ID Submetter2: 038163630

In [58]:
#///////////////////////lab 2.1///////////////////////
import time
import math
import numpy as np
import pdb
import operator
from __future__ import division, print_function
from random import randint, random
import sys
from sklearn.cluster import KMeans


CRED = '\033[91m'
CGREEN = '\033[92m'
CFLASH = '\033[5m'
CBOLD = '\33[1m'
CITALIC = '\33[3m'
CEND = '\033[0m'

GA_ELITRATE = 0.1 # elitism rate
GA_MAXITER = 16384 # maximum iterations
GA_POPSIZE = 100 # ga population size
GA_TARGET = "Hello World!"
GA_MUTATIONRATE = 0.25 #  mutation rate
GA_MAXMUTATIONRATE = 0.75 # max mutation rate
GA_MINMUTATIONRATE = 0.1 #  min mutation rate
t=0 #genration number
rate=0.5
IMMIGRANTSNUM = 25 # number of individuals to replace - Random Immigrants

xrange=range
linearMutation=(GA_MAXMUTATIONRATE - GA_MINMUTATIONRATE) / GA_MAXITER

class Agent:
    def __init__(self, string, fitness):
        self.string = string
        self.fitness = fitness
        self.age = 0
        self.species = -1

'''def roulette_selection(weights):
      #  performs weighted selection or roulette wheel selection on a list
        and returns the index selected from the list

        # sort the weights in ascending order
        sorted_indexed_weights = sorted(enumerate(weights), key=operator.itemgetter(1));
        indices, sorted_weights = zip(*sorted_indexed_weights);
        # calculate the cumulative probability
        tot_sum=sum(sorted_weights)
        prob = [x/tot_sum for x in sorted_weights]
        cum_prob=np.cumsum(prob)
        # select a random a number in the range [0,1]
        random_num=random()

        for index_value, cum_prob_value in zip(indices,cum_prob):
            if random_num < cum_prob_value:
                return index_value '''

def initPop(population, buffer):
    #initialize population
    for i in range(GA_POPSIZE):
        randString = "".join([chr(randint(32, 122)) for x in range(len(GA_TARGET))])
        population.append(Agent(randString, 0))
        buffer.append(Agent("", 0))

def calcPopulationFitness(population):
    #calculate fitness (letter distance)
    for i in population:
        fitness = 0
        for j in range(len(GA_TARGET)):
            fitness += abs(ord(i.string[j]) - ord(GA_TARGET[j]))
        i.fitness = fitness
   # print("fitness", fitness)    

def calcAverage(population):
 # calculating the Average and Standard Deviation
    sum_1=0
    for i in population:
        sum_1+= i.fitness

    avg = sum_1/len(population)
    
    sum_2=0
    for i in population:
        sum_2+= math.pow(i.fitness-avg,2)

    s_deviation= math.sqrt(sum_2/len(population))

    print(CEND + "Average:" + CGREEN, avg, CEND + "Standard Deviation:" + CGREEN, s_deviation)

    SP=population[0].fitness/avg
    print("selection pressure" , SP)


def singlePointCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        spos = randint(0, target_size - 1)
       
        buffer[i].string = population[i1].string[:spos] + population[i2].string[spos:]
    return buffer

def twoPointCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        spos1 = randint(0, target_size - 2)
        spos2 = randint(spos1+1, target_size - 1)
       
        buffer[i].string = population[i1].string[:spos1] + population[i2].string[spos1:spos2] + population[i1].string[spos2:]
    return buffer

def uniformCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        uni = randint(1, 2)
        if uni==1:
          buffer[i].string = population[i1].string
        else:
          buffer[i].string = population[i2].string
    return buffer
    
def elitism(population, buffer, esize):
    return [Agent(x.string, x.fitness) for x in population[:esize]] + [x for x in buffer[esize:]]

def mutate(agent):
    target_size = len(GA_TARGET)
    ipos = randint(0, target_size - 1)
    delta = randint(32, 122)
    agent.string = agent.string[:ipos] + chr((ord(agent.string[ipos]) + delta) % 122) + agent.string[ipos + 1 : ]

def mate(population, buffer, choiceFunc, crossOverFunc):
    esize = int(GA_POPSIZE * GA_ELITRATE)
    target_size = len(GA_TARGET)

    buffer = choiceFunc(population, buffer, esize)
    buffer = crossOverFunc(population, buffer, esize, len(GA_TARGET))

    # Mate the rest
    for i in range(esize, GA_POPSIZE):
        if random() < GA_MUTATIONRATE:
            mutate(buffer[i])
    return buffer


def mateLinear(population, buffer, choiceFunc, crossOverFunc):
    esize = int(GA_POPSIZE * GA_ELITRATE)
    target_size = len(GA_TARGET)

    buffer = choiceFunc(population, buffer, esize)
    buffer = crossOverFunc(population, buffer, esize, len(GA_TARGET))

    # Mate the rest
    GA_MUTATIONRATE=GA_MAXMUTATIONRATE
    for i in range(esize, GA_POPSIZE):
        GA_MUTATIONRATE=GA_MUTATIONRATE-linearMutation
        if random() < GA_MUTATIONRATE:
            mutate(buffer[i])

    return buffer 

def mateAdaptive(population, buffer, choiceFunc, crossOverFunc):
    esize = int(GA_POPSIZE * GA_ELITRATE)
    target_size = len(GA_TARGET)

    buffer = choiceFunc(population, buffer, esize)
    buffer = crossOverFunc(population, buffer, esize, len(GA_TARGET))
    GA_MUTATIONRATE = (2 * (GA_MAXMUTATIONRATE ** 2) * (math.e ** (rate * t)))/(GA_MAXMUTATIONRATE + (GA_MAXMUTATIONRATE * (math.e ** (rate*t))))
    return buffer 
       

def tournamentSelection(population, buffer, size):
    Selections = []         # array of selections
    for i in range(size):
        pick1 = population[randint(0,GA_POPSIZE-1)]     # pick first genom
        pick2 = population[randint(0,GA_POPSIZE-1)]     # pick second genom

        if pick1.fitness < pick2.fitness:   # find better genom
            Selections.append(Agent(pick1.string, pick1.fitness))
        else:
            Selections.append(Agent(pick2.string, pick2.fitness))

    return Selections + [i for i in buffer[size:]]


def SUS(population, buffer, size):
    #single spin of Roulette wheel
    selections = []
    fitnesses = [(1/agent.fitness) for agent in population]
    index = roulette_selection(fitnesses)
    selections.append(population[index])

    return selections      

def RWS(population, buffer, size):
    #Roulette wheel selection
    selections = []
    fit = [(1/agent.fitness) for agent in population]
    for i in range(size):
        index = roulette_selection(fit)
        selections.append(population[index])

    return selections + [i for i in buffer[size:]] 

def ranking(population, buffer, size):
  # Get the number of individuals in the population.
  #size
  # Use the gauss formula to get the sum of all ranks (sum of integers 1 to N).
  rank_sum = size * (size + 1) / 2
  # Sort and go through all individual fitnesses; enumerate ranks from 1.
  for rank, ind_fitness in enumerate(sorted(population), 1):
    yield rank, ind_fitness, float(rank) / rank_sum  
  # Read the fitnesses from the command line arguments.
        #population.sort(key= lambda x: x.fitness)  # sort population array by fitness
  # Iterate through the computed tuples and print the table rows.

def aging(population, buffer, size):
    adultAge = 6
    for agent in population:
        agent.fitness += pow((agent.age-adultAge),2)   #adding age bonus
    return RWS(population, buffer, size)

def bullsEye(population):
    for i in population:
        fitness = 0
        for j in range(len(GA_TARGET)):
          #if it's bullsEye 
            if i.string[j] == GA_TARGET[j]:  
                fitness+=0
              #if it's not bullsEye but the letter is correct 
            elif i.string[j] in GA_TARGET: 
                fitness+=30
                # if the letter is not correct 
            else:               
                fitness+=80
                
        i.fitness = fitness    

def levenshteinDistance(s, t, costs=(1, 1, 1)):

    rows = len(s)+1
    cols = len(t)+1
    deletes, inserts, substitutes = costs
    
    dist = [[0 for x in range(cols)] for x in range(rows)]

    # source prefixes can be transformed into empty strings 
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = row * deletes

    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = col * inserts
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = substitutes
            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + cost) # substitution
    
 
    return dist[row][col]



def kendalltau(p1, p2):
    n = len(p1)
    index = [None] * n
    for i in rang(n):
        v=p2[i]; index[v] = i

    dis=0 #raw distance = number pair mis-orderings 
    for i in range(n):
       for j in range(i+1, n):
           if index[p1[i]] > index[p1[j]]:
               dis+=1
    numpairs = n * (n - 1)/2.0
    normaldis=dis / numpairs
    return(dis, normaldis)
    

def geneticDiversity(population):
    diversity = 0
    for i in range(len(population)):
        for j in range(len(population)):
            diversity += levenshteinDistance(population[i].string, population[j].string)
    genetic_diversity =  diversity / len(population)
    print("Genetic Diversity: " ,genetic_diversity)

def thresholdSpeciation(population):    
    threshold = GA_POPSIZE * 0.25 #try diffrent value  
    species_count=30
    speciation = [[]] * 30
    for i in range(len(population)):
        check = 0
        for j in range(len(speciation)):
            for x in range(len(speciation[j])):
                if levenshteinDistance(population[i].string, speciation[j][x].string) <= threshold:
                   check += 1
        if check == len(speciation[j]):
            speciation[j].append(population[i])
            population[i].species = j
    if population[i].species == -1:
        population[i].species = species_count
        species_count += 1
        speciation.append([population[i]])  
    print(species_count)


def clustringSpeciation(population):
    wcss=[]
    x = []
    for i in range(len(population)):
      x.append(population[i].fitness)
    kmeans = KMeans(2)
    data = np.array(x)
    X = data.reshape(-1, 1)
    kmeans.fit(X)
    wcss_iter = kmeans.inertia_
    wcss.append(wcss_iter)
    number_clusters = range(len(population))
    #print(number_clusters)

def randomImmigrants(population):
    population.sort(key=lambda x: x.fitness,reverse=True)
    total = len(population) - IMMIGRANTSNUM
    new_population = population[:total]
    for _ in range(IMMIGRANTSNUM):
       randString = "".join([chr(randint(32, 122)) for x in range(len(GA_TARGET))])
       new_population.append(Agent(randString, 0))  
    calcPopulationFitness(new_population)
    return new_population

def generate_genotype():
    """ Function that genarates the genotype of the individual """
    gen = []
    for _ in range(GA_POPSIZE):
        gen.append(randint(0,1))
    return 

if __name__ == "__main__":
    starting_time = time.time()
    population = []
    buffer = []
    #values = []
    initPop(population, buffer)

    for i in range(GA_MAXITER):
        int_clock_ticks = time.time()
        t=t+1
        
        #calcPopulationFitness(population)
        bullsEye(population)
       # randomImmigrants(population)
        population.sort(key= lambda x: x.fitness)  # sort population array by fitness
        print(CBOLD + "Best:" + CGREEN, population[0].string,CEND + "fitness: " + CGREEN,population[0].fitness)     # print string with best fitness
        calcAverage(population)
        #geneticDiversity(population)
        #thresholdSpeciation(population)
        clustringSpeciation(population)
       # print("levenshtein distance: ", LD(population[0].string,GA_TARGET))
        #print("kendalltau distance: ", kendalltau(population[0].string,GA_TARGET))
        #values[0]= population[0].fitness

        if population[0].fitness==0:
            print(CEND+ "Clock ticks time: " + CGREEN, time.time()-int_clock_ticks)
            print("")
            break
        
      #  buffer = mate(population, buffer, tournamentSelection, twoPointCrossOver)  # mate
        buffer = mateLinear(population, buffer, tournamentSelection, twoPointCrossOver)  #linear mate
       # buffer = mateAdaptive(population, buffer, tournamentSelection, twoPointCrossOver) # Adaptive mate
        population, buffer = buffer, population


        for agent in population:
            agent.age+=1
            
        print(CEND + "Clock ticks time: " + CRED, time.time()-int_clock_ticks)
        print(""+ CEND)

    #x = np.quantile(values, [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])

    #print(x)

    print(CBOLD + "Total time Elapsed: " + CGREEN, time.time()-starting_time)

Best: :ec#l$WJd"eM fitness:  650
Average: 900.4 Standard Deviation: 59.108713401663515
selection pressure 0.7219013771657041
Clock ticks time:  0.017676115036010742

Best: :ec`E$WJd"eM fitness:  700
Average: 860.7 Standard Deviation: 60.36977720681097
selection pressure 0.8132915069129778
Clock ticks time:  0.018932104110717773

Best: nyW8FrAdH^le fitness:  660
Average: 807.3 Standard Deviation: 53.42012729299703
selection pressure 0.8175399479747306
Clock ticks time:  0.015995264053344727

Best: w<l'oaW X<rE fitness:  620
Average: 765.9 Standard Deviation: 50.00189996390135
selection pressure 0.8095051573312443
Clock ticks time:  0.01994490623474121

Best: nyloG$WJH^le fitness:  600
Average: 744.8 Standard Deviation: 58.9657527722661
selection pressure 0.8055853920515575
Clock ticks time:  0.018827199935913086

Best: NQlQoyW 0Hl7 fitness:  570
Average: 709.5 Standard Deviation: 51.52426612771889
selection pressure 0.8033826638477801
Clock ticks time:  0.015503883361816406

Best: o lwE

In [ ]:
#///////////////////////Part 2.2///////////////////////
import time
import math
import numpy as np
import pdb
import operator
from __future__ import division, print_function
from random import randint, random, randrange
import sys
from matplotlib import pyplot as plt


CRED = '\033[91m'
CGREEN = '\033[92m'
CFLASH = '\033[5m'
CBOLD = '\33[1m'
CITALIC = '\33[3m'
CEND = '\033[0m'

GA_ELITRATE = 0.1 # elitism rate
GA_MAXITER = 1000 # maximum iterations
GA_POPSIZE = 1000 # ga population size
GA_TARGETPATERN = ""
GA_MUTATIONRATE = 0.25 #  mutation rate
GA_ELITRATE = 0.1  # elitism rate
GA_MUTATION = sys.maxsize * GA_MUTATIONRATE
NUMBITS =20


class Agent:   
    def __init__(self, string, fitness):
        self.string = string
        self.fitness = fitness
        self.age = 0

def roulette_selection(weights):
      #  performs weighted selection or roulette wheel selection on a list
       # and returns the index selected from the list

        # sort the weights in ascending order
        sorted_indexed_weights = sorted(enumerate(weights), key=operator.itemgetter(1));
        indices, sorted_weights = zip(*sorted_indexed_weights);
        # calculate the cumulative probability
        tot_sum=sum(sorted_weights)
        prob = [x/tot_sum for x in sorted_weights]
        cum_prob=np.cumsum(prob)
        # select a random a number in the range [0,1]
        random_num=random()

        for index_value, cum_prob_value in zip(indices,cum_prob):
            if random_num < cum_prob_value:
                return index_value

def initPop(population, buffer):
    #initialize population
    for i in range(GA_POPSIZE):
        randString = "".join([chr(randint(32, 122)) for x in range(len(GA_TARGETPATERN))])
        population.append(Agent(randString, 0))
        buffer.append(Agent("", 0))

def initializePattern():
    bitstr = ""
    for i in range(NUMBITS):
        pick = randrange(0, 2)
        if pick == 1:
            bitstr += '1'
        else:
            bitstr += '0'
    return bitstr 
    

def initializeBit():
    pick = randrange(0, 4)
    if pick == 0:
        bit = '0'
    elif pick == 1:
        bit = '1'
    else:
        bit = '?'
    #print(bit)
    return bit

def initGenome():
    bitstring = ""
    for i in range(NUMBITS):
        bitstring +=initializeBit()
    #print(bitstring)
    return bitstring  

def initPopulation(population):
    print(len(population))  
    for i in range(len(population)):
        citizen = Agent([], 0)
        citizen.string = initGenome()
        population[i] = citizen
    return population  

def countLearned(string):
    count = 0
    for i in range(len(string)):
        y = string[i]
        if y == '?':
            count += 1
    return count            


def calcPopulationFitness(population):
    #calculate fitness (letter distance)
    for i in population:
        fitness = 0
        for j in range(len(GA_TARGETPATERN)):
            fitness += abs(ord(i.string[j]) - ord(GA_TARGETPATERN[j]))
        i.fitness = fitness

def randomReplace():
    rand = randrange(0, 2)
    if rand == 0:
        bit = '0'
    else:
        bit = '1'
    return bit

def correctPositions(target, string):
    count = 0
    for i in range(len(string)):
        x = target[i]
        y = string[i]
        if y == x:
            count += 1

    return count

def incorrectPositions(target, string):
    count = 0
    for i in range(len(string)):
        x = target[i]
        y = string[i]
        if y != "?":
            if x != y:
                count += 1
    return count 



def calcAverage(population):
 # calculating the Average and Standard Deviation
    sum_1=0
    for i in population:
        sum_1+= i.fitness

    avg = sum_1/len(population)
    
    sum_2=0
    for i in population:
        sum_2+= math.pow(i.fitness-avg,2)

    s_deviation= math.sqrt(sum_2/len(population))

    print(CEND + "Average:" + CGREEN, avg, CEND + "Standard Deviation:" + CGREEN, s_deviation)

def singlePointCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        spos = randint(0, target_size - 1)
       
        buffer[i].string = population[i1].string[:spos] + population[i2].string[spos:]
    return buffer

def twoPointCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        spos1 = randint(0, target_size - 2)
        spos2 = randint(spos1+1, target_size - 1)
       
        buffer[i].string = population[i1].string[:spos1] + population[i2].string[spos1:spos2] + population[i1].string[spos2:]
    return buffer

def uniformCrossOver(population,buffer,esize,target_size):
    for i in range(esize, GA_POPSIZE):
        i1 = randint(0, int(GA_POPSIZE/2) - 1)
        i2 = randint(0, int(GA_POPSIZE/2) - 1)
        uni = randint(1, 2)
        if uni==1:
          buffer[i].string = population[i1].string
        else:
          buffer[i].string = population[i2].string
    return buffer

def sortFitness(population: list):
        population.sort(key=lambda x: x.fitness)
        return population    

def elitism(population, buffer, esize):
    return [Agent(x.string, x.fitness) for x in population[:esize]] + [x for x in buffer[esize:]]

def mutate(agent):
    target_size = len(GA_TARGETPATERN)
    ipos = randint(0, target_size - 1)
    delta = randint(32, 122)
    agent.string = agent.string[:ipos] + chr((ord(agent.string[ipos]) + delta) % 122) + agent.string[ipos + 1 : ]

#def mate(population, buffer, choiceFunc, crossOverFunc):
#def mate(population, buffer):
 #   esize = int(GA_POPSIZE * GA_ELITRATE)
  #  target_size = len(GA_TARGETPATERN)

    #buffer = choiceFunc(population, buffer, esize)
    #buffer = crossOverFunc(population, buffer, esize, len(GA_TARGETPATERN))

    # Mate the rest
   # for i in range(esize, GA_POPSIZE):
   #     if random() < GA_MUTATIONRATE:
    #        muate(buffer[i])
    #return buffer
def mate(population, buffer):
    size = int(GA_POPSIZE* GA_ELITRATE)
    elitism(population, buffer, size)
    tsize = len(GA_TARGETPATERN)
    indx1 = -1
    indx2 = -2
    for i in range(len(population)):
        indx1 = tournamentSelection(population ,buffer, size)
        indx2 = tournamentSelection(population ,buffer, size)
        #indx1 = random.randrange(0, self.P_size // 2)
        #indx2 = random.randrange(0, self.P_size // 2)

        # single crossover
        pos = randrange(0, tsize)
        buffer[i] = Agent(indx1[0: pos] + indx2[pos:], 0)
        #buffer[i] = population[indx1]

    return buffer, population    
  

def tournamentSelection(population, buffer, size):
    Selections = []         # array of selections
    for i in range(size):
        pick1 = population[randint(0,GA_POPSIZE-1)]     # pick first genom
        pick2 = population[randint(0,GA_POPSIZE-1)]     # pick second genom

        if pick1.fitness < pick2.fitness:   # find better genom
            Selections.append(Agent(pick1.string, pick1.fitness))
        else:
            Selections.append(Agent(pick2.string, pick2.fitness))

    return Selections + [i for i in buffer[size:]]


def SUS(population, buffer, size):
    #single spin of Roulette wheel
    selections = []
    fitnesses = [(1/agent.fitness) for agent in population]
    index = roulette_selection(fitnesses)
    selections.append(population[index])

    return selections      

def RWS(population, buffer, size):
    #Roulette wheel selection
    selections = []
    fit = [(1/agent.fitness) for agent in population]
    for i in range(size):
        index = roulette_selection(fit)
        selections.append(population[index])

    return selections + [i for i in buffer[size:]] 

def ranking(population, buffer, size):
  # Get the number of individuals in the population.
  #size
  # Use the gauss formula to get the sum of all ranks (sum of integers 1 to N).
  rank_sum = size * (size + 1) / 2
  # Sort and go through all individual fitnesses; enumerate ranks from 1.
  for rank, ind_fitness in enumerate(sorted(population), 1):
    yield rank, ind_fitness, float(rank) / rank_sum  
  # Read the fitnesses from the command line arguments.
        #population.sort(key= lambda x: x.fitness)  # sort population array by fitness
  # Iterate through the computed tuples and print the table rows.

def aging(population, buffer, size):
    adultAge = 6
    for agent in population:
        agent.fitness += pow((agent.age-adultAge),2)   #adding age bonus
    return RWS(population, buffer, size)

def bullsEye(population):
    for i in population:
        fitness = 0
        for j in range(len(GA_TARGETPATERN)):
          #if it's bullsEye 
            if i.string[j] == GA_TARGETPATERN[j]:  
                fitness+=0
              #if it's not bullsEye but the letter is correct 
            elif i.string[j] in GA_TARGETPATERN: 
                fitness+=30
                # if the letter is not correct 
            else:               
                fitness+=80
                
        i.fitness = fitness 
def localSearch(population, correct, incorrect, learned):

    fitness = [0] * GA_POPSIZE
    print(CBOLD +"Target: ",GA_TARGETPATERN)

    for i in range(GA_POPSIZE):  # iterate over the population
        tmp_iteration = GA_MAXITER
        inner_correct = [0] * 1000
        inner_incorrect = [0] * 1000
        for j in range(GA_MAXITER):  # for each genome iterate over max iterations
            inner_correct[j] = correctPositions(GA_TARGETPATERN, population[i].string) / len(GA_TARGETPATERN)
            inner_incorrect[j] = incorrectPositions(GA_TARGETPATERN, population[i].string) / len(GA_TARGETPATERN)

            bitstr = ""
            old_str = population[i].string
            for k in range(NUMBITS):
                if population[i].string[k] == '?':
                    # if it is '?' then make it 0 with p = 0.5 and 1 with p = 0.5
                    bitstr += randomReplace()
                else:
                    bitstr += population[i].string[k]

            population[i].string = bitstr

      #      if i == 1:
      #          print(population[i].string)
            if population[i].string == GA_TARGETPATERN:
                tmp_iteration = j + 1
                print(CGREEN + "Eureka!! :", population[i].string)
                break
            population[i].string = old_str

        correct[i] = sum(inner_correct) / tmp_iteration
        incorrect[i] = sum(inner_incorrect) / tmp_iteration
        learned[i] = countLearned(population[i].string) / len(GA_TARGETPATERN)

        n = GA_MAXITER - tmp_iteration
        fitness[i] = 1 + (19 * n / GA_MAXITER)  # fitness = 1 + 19n/1000
        population[i].fitness = fitness[i]
    return max(fitness), fitness, population, correct, incorrect, learned               
    

if __name__ == "__main__":
    starting_time = time.time()
    generationsNum=40
    population = [None] *1000
    buffer = [None] *1000
    #values = []
    #initPop(population, buffer)
    GA_TARGETPATERN = initializePattern() #makes a bit pattern of 20 bits
    population =initPopulation(population)
    
    correct = [0] * len(population)
    incorrect = [0] * len(population)
    learned = [0] * len(population)
    x = [0] * generationsNum
    AVG_correct_gen = [0] * generationsNum
    AVG_incorrect_gen = [0] * generationsNum
    AVG_learned_gen = [0] * generationsNum

    for i in range(GA_MAXITER):
        int_clock_ticks = time.time()
 #       x[i] = i
    #    print("itreation:", i)        
        #calcPopulationFitness(population)
      #  bullsEye(population)
        population.sort(key= lambda x: x.fitness)  # sort population array by fitness
       # print(CBOLD + "Best:" + CGREEN, population[0].string,CEND + "fitness: " + CGREEN,population[0].fitness)     # print string with best fitness
        #calcAverage(population)
        #values[0]= population[0].fitness
      #  if population[0].fitness==0:
       #     print(CEND+ "Clock ticks time: " + CGREEN, time.time()-int_clock_ticks)
        #    print("")
         #   break
        print(CEND + "Clock ticks time: " + CRED, time.time()-int_clock_ticks)
        print(""+ CEND)
    #x = np.quantile(values, [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])
    #print(x)
        max_fitness, fitness, population, correct, incorrect, learned = localSearch(population, correct, incorrect, learned)
        Avg_correct = sum(correct) / len(correct)
        Avg_incorrect = sum(incorrect) / len(incorrect)
        Avg_learned = sum(learned) / len(learned)
        #population, buffer = mate(population, buffer,tournamentSelection,twoPointCrossOver)
        population, buffer = mate(population, buffer)
        print(CBOLD + "Average correct:" + CGREEN, Avg_correct)
        print(CBOLD + "Average incorrect:" + CGREEN, Avg_incorrect)
        print(CBOLD + "Average learned"+ CGREEN, Avg_learned)
        AVG_correct_gen[i] = Avg_correct
        AVG_incorrect_gen[i] = Avg_incorrect
        AVG_learned_gen[i] = Avg_learned
  #  print("fitness:", fitness)
        print(CBOLD + "max fitness:" + CGREEN, max_fitness)
        break
    print(CBOLD + "Total time Elapsed: " + CGREEN, time.time()-starting_time)
      
 
   

      #  population = problem.sort_by_fitness(population)
        #fitness = [0] * problem.P_size

        

        

1000
Clock ticks time:  0.00013566017150878906

Target:  11001010111001111101
Eureka!! : 11001010111001111101
Eureka!! : 11001010111001111101
Average correct: 0.24510000000000076
Average incorrect: 0.25305000000000066
Average learned 0.5007499999999998
max fitness: 15.725
Total time Elapsed:  28.304070711135864
